# Task 4: Optimize Portfolio Based on Forecast

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima

# Load the cleaned data
tsla_df = pd.read_csv('../data/TSLA_cleaned.csv', index_col='Date', parse_dates=True)
bnd_df = pd.read_csv('../data/BND_cleaned.csv', index_col='Date', parse_dates=True)
spy_df = pd.read_csv('../data/SPY_cleaned.csv', index_col='Date', parse_dates=True)

## 1. Calculate Expected Returns

In [ ]:
# TSLA: Use the ARIMA forecast to get the expected return
stepwise_fit = auto_arima(tsla_df['Close'], trace=False, suppress_warnings=True) # Re-running auto_arima to get the best order
p, d, q = stepwise_fit.order
model = ARIMA(tsla_df['Close'], order=(p, d, q))
model_fit = model.fit()
forecast = model_fit.forecast(steps=252) # 1 year forecast
expected_return_tsla = (forecast.iloc[-1] - tsla_df['Close'].iloc[-1]) / tsla_df['Close'].iloc[-1]

In [ ]:
# BND and SPY: Use historical average daily returns (annualized)
returns_bnd = bnd_df['Close'].pct_change().dropna()
returns_spy = spy_df['Close'].pct_change().dropna()
expected_return_bnd = returns_bnd.mean() * 252
expected_return_spy = returns_spy.mean() * 252

In [ ]:
expected_returns = np.array([expected_return_tsla, expected_return_bnd, expected_return_spy])
print(f'Expected Annual Returns: TSLA: {expected_return_tsla:.2%} BND: {expected_return_bnd:.2%} SPY: {expected_return_spy:.2%}')

## 2. Calculate Covariance Matrix

In [ ]:
returns_tsla = tsla_df['Close'].pct_change().dropna()
returns_df = pd.concat([returns_tsla, returns_bnd, returns_spy], axis=1).dropna()
returns_df.columns = ['TSLA', 'BND', 'SPY']
cov_matrix = returns_df.cov() * 252

## 3. Generate the Efficient Frontier

In [ ]:
num_portfolios = 10000
results = np.zeros((3, num_portfolios))
weights_record = []
for i in range(num_portfolios):
    weights = np.random.random(3)
    weights /= np.sum(weights)
    weights_record.append(weights)
    portfolio_return = np.sum(weights * expected_returns)
    portfolio_stddev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    results[0,i] = portfolio_return
    results[1,i] = portfolio_stddev
    results[2,i] = (portfolio_return - 0.02) / portfolio_stddev # Sharpe Ratio, assuming 2% risk-free rate

results_frame = pd.DataFrame(results.T, columns=['ret','stdev','sharpe'])
results_frame['weights'] = weights_record

## 4. Identify Key Portfolios and Plot the Efficient Frontier

In [ ]:
max_sharpe_port = results_frame.iloc[results_frame['sharpe'].idxmax()]
min_vol_port = results_frame.iloc[results_frame['stdev'].idxmin()]

In [ ]:
plt.figure(figsize=(14, 7))
plt.scatter(results_frame.stdev, results_frame.ret, c=results_frame.sharpe, cmap='viridis')
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Return')
plt.scatter(max_sharpe_port['stdev'], max_sharpe_port['ret'], marker='*', color='r', s=500, label='Maximum Sharpe Ratio')
plt.scatter(min_vol_port['stdev'], min_vol_port['ret'], marker='*', color='g', s=500, label='Minimum Volatility')
plt.title('Efficient Frontier')
plt.legend()
plt.show()

## 5. Recommend an Optimal Portfolio

In [ ]:
print("Maximum Sharpe Ratio Portfolio:")
print(f"Weights: TSLA={max_sharpe_port['weights'][0]:.2%}, BND={max_sharpe_port['weights'][1]:.2%}, SPY={max_sharpe_port['weights'][2]:.2%}")
print(f"Expected Annual Return: {max_sharpe_port['ret']:.2%}")
print(f"Expected Annual Volatility: {max_sharpe_port['stdev']:.2%}")
print(f"Sharpe Ratio: {max_sharpe_port['sharpe']:.2f}")

print("Minimum Volatility Portfolio:")
print(f"Weights: TSLA={min_vol_port['weights'][0]:.2%}, BND={min_vol_port['weights'][1]:.2%}, SPY={min_vol_port['weights'][2]:.2%}")
print(f"Expected Annual Return: {min_vol_port['ret']:.2%}")
print(f"Expected Annual Volatility: {min_vol_port['stdev']:.2%}")
print(f"Sharpe Ratio: {min_vol_port['sharpe']:.2f}")

### Recommendation
For an investor seeking the highest risk-adjusted return, the **Maximum Sharpe Ratio Portfolio** is the recommended choice. This portfolio provides the best return for the amount of risk taken.

For a more risk-averse investor, the **Minimum Volatility Portfolio** would be more suitable, as it is designed to minimize risk, albeit with a lower expected return.